In [1]:
# Get raw data
with open('input/11.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1 & 2
n_types = 5
elids = dict(zip((z:=sorted({i.split()[0] for i in rawinput.replace('-', ' ').split(' a ')[1:]})), range(n_types)))

poss_locs = np.zeros((1, n_types, 2), dtype=int)
for i,j in enumerate(rawinput.split('\n')):
    k = j[:-1].replace(' and ', ', ').split(' contains ')[1]
    for l in k.split(','):
        if l and not 'nothing' in l:
            poss_locs[0, elids[l.strip()[2:].replace('-', ' ').split()[0]], int(l.split()[-1] == 'microchip')] = i

# For Part 2
poss_locs = np.append(poss_locs, np.zeros((1, 2, 2), dtype=int), axis=1)
n_types = 7

poss_eloc = np.zeros((1), dtype=int)

exp_ix = np.array([[d, i%n_types,i//n_types,j%n_types,j//n_types] 
                   for d in [-1, 1] for i in range(n_types*2) for j in range(i, n_types*2)])

def get_scenid(poss_locs, poss_eloc):
    return np.char.add(np.sort(poss_locs.astype('<U1').reshape(-1, n_types*2).view('<U2'), axis=1)
                            .reshape(-1).view(f'<U{n_types*2}'),
                       poss_eloc.astype('<U1'))

history = get_scenid(poss_locs, poss_eloc).tolist()
nsteps = 0

while True:
    nsteps += 1
    
    # Expand possible moves
    ix = np.repeat(np.arange(poss_locs.shape[0]), exp_ix.shape[0])
    ix2 = np.tile(exp_ix, (poss_locs.shape[0], 1))
    filt = ((poss_eloc[ix]+ix2[:, 0] >= 0) 
            & (poss_eloc[ix]+ix2[:, 0] < 4) 
            & (poss_locs[ix, ix2[:, 1], ix2[:, 2]] == poss_eloc[ix]) 
            & (poss_locs[ix, ix2[:, 3], ix2[:, 4]] == poss_eloc[ix]))

    poss_eloc = poss_eloc[ix[filt]]+ix2[filt, 0]
    poss_locs = poss_locs[ix[filt]]
    poss_locs[np.arange(poss_locs.shape[0]), ix2[filt, 1], ix2[filt, 2]] = poss_eloc
    poss_locs[np.arange(poss_locs.shape[0]), ix2[filt, 3], ix2[filt, 4]] = poss_eloc
    
    # Filter illegal moves (fried chips)
    filt = np.all((poss_locs[:, :, 0] == poss_locs[:, :, 1]) 
                  | (~np.any(np.expand_dims(poss_locs[:, :, 0], 1) 
                             == np.expand_dims(poss_locs[:, :, 1], 2), axis=2)),
                  axis=1)    
    poss_locs = poss_locs[filt]
    poss_eloc = poss_eloc[filt]
        
    # Filter equivalent scenarios
    scenid = get_scenid(poss_locs, poss_eloc)
    filt = (scenid[sortix:=np.argsort(scenid)] != np.append([''], scenid[sortix][:-1]))[np.argsort(sortix)]
    poss_locs = poss_locs[filt]
    poss_eloc = poss_eloc[filt]
        
    # Filter scenarios already seen
    scenid = scenid[filt]
    filt = ~np.isin(scenid, history)
    poss_locs = poss_locs[filt]
    poss_eloc = poss_eloc[filt]
    
    # Update history list
    history += scenid[filt].tolist()
    
    if np.any(np.all(poss_locs == 3, axis=(1,2))):
        break

nsteps

71